## Setup 

### Requirements
For this exercise we'll need Bitcoin Core. This notebook has been tested with [v24.0.1](https://github.com/bitcoin/bitcoin/releases/tag/v24.0.1).

Below, set the paths for:
1. The bitcoin core functional test framework directory.
2. The directory containing bitcoin-tx-tutorial.

**You'll need to edit these next two lines for your local setup.**

In [1]:
path_to_bitcoin_functional_test = "/Users/dariuscognac/bitcoin/test/functional"
path_to_bitcoin_tx_tutorial = "/Users/dariuscognac/Documents/Github/bitcoin-tx-tutorial"

import sys

# Add the functional test framework to our PATH
sys.path.insert(0, path_to_bitcoin_functional_test)
from test_framework.test_shell import TestShell

# Add the bitcoin-tx-tutorial functions to our PATH
sys.path.insert(0, path_to_bitcoin_tx_tutorial)
from functions import *
from functions.bip_0340_reference import *

# Elliptic Curve Math Review

Elliptic Curve math involves scalars and points.

* A scalar is a positive integer which is smaller than the group order.
* A point lies on the curve and is denoted by a pair of co-ordinates (eg `(x,y)`).

In Bitcoin, key pair generation and signing is performed over the secp256k1 curve. All scalars are modulo the group order `SECP256K1_ORDER`, which is a very large number

![test](../images/taproot-workshop/ec_math0.jpg)

_An overview of all operations of scalars and points over elliptic curves._

## Elliptic curve math in python

**`Scalars`:** All Scalar operations over secp256k1 can be performed with python integers and the modulo `%` operator. 

Scalar addition, subtraction, multiplication and division over secp256k1 are modulo a large prime number SECP256K1_ORDER.

* All scalar operations are performed modulo `SECP256K1_ORDER`.
* Addition: `(a + b) % SECP256K1_ORDER`
* Subtraction: `-a = SECP256K1_ORDER - a`
* Multiplication: `(a * b) % SECP256K1_ORDER`
* Division (see [Fermat's little theorem](https://en.wikipedia.org/wiki/Fermat%27s_little_theorem)): `1/b = b ** (SECP256K1_ORDER-2) % SECP256K1_ORDER`

**`Points`:** The [BIP340 reference code](https://github.com/bitcoin/bips/blob/master/bip-0340/reference.py) provides a `Point` class for and some helpful functions for performing point addition (e.g. for adding two pubkeys together) and multiplication (e.g. multiplying the generator point by a private key to get a pubkey).

```
Point = Tuple[int, int]
```

* Addition: `point_add(P1: Optional[Point], P2: Optional[Point]) -> Optional[Point]`
* Multiplication: `point_mul(P: Optional[Point], n: int) -> Optional[Point]`

## Elliptic Curves in Bitcoin

### Sekp256k1
Secp256k1 refers to the set of parameters used in Bitcoin's elliptic curve cryptography. A full list of the parameters can be found [here](https://en.bitcoin.it/wiki/Secp256k1). Two parameters which we will use in a few places (in particular for constructing taproot outputs) are the order, SECP256K1, and the generator point, G.

### Private keys
Private keys in Bitcoin are 32 bytes scalars. Note that private keys must be a value between 1 and SECP256K1.

In [2]:
privkey = bytes.fromhex("0000000000000000000000000000000000000000000000000000000000000001")

### Public keys
Public keys are points on the elliptic curve and have an x and y coordinate. A public key is generated by multiplying the generator point by the private key. We can use the BIP340 function `point_mul` for this operation. Note that it expects the private key to be an integer:

In [3]:
privkey_int = int_from_bytes(privkey)
pubkey = point_mul(G, privkey_int)
print("pubkey: ", pubkey)

pubkey:  (55066263022277343669578718895168534326250603453777594175500187360389116729240, 32670510020758816978083085130507043184471273380659243275938904335757337482424)


**Exercise**: You'll notice the pubkey has two integers representing an x and y coordinate on the elliptic curve. Try converting these values to hex (you can use `bytes_from_int()`) and compare it to the generator point (G) listed above. What do you notice, and why is this the case?

**Answer**: 
```
bytes_from_int(pubkey[0]).hex()
bytes_from_int(pubkey[1]).hex()
```
The pubkey is the same as the generator point because we chose a private key of 1, so when we multiply them together to get the public key (G\*1=G) we end up with the same value.

#### Uncompressed/Compressed public keys
Before taproot, Bitcoin used either 65-byte uncompressed, or 33-byte compressed public keys. Interestingly, the only reason uncompressed public keys were more common in the earlier days (~2012) was because because compressed public keys were poorly documented in OpenSSL.
- Uncompressed public keys: `0x04` followed by two 256-bit integers for the x and y-coordinates
- Compressed public keys: a prefix of either `0x02` or `0x03` depending on whether the y-coordinate was even or odd, respectively, followed by a 256-bit integer for the x coordinate. 

In [4]:
# using the same privkey value from above
pubkey = point_mul(G, privkey_int)

# constant `04` prefix followed by x and y coordinates 
uncompressed = b"\x04" + bytes_from_int(pubkey[0]) + bytes_from_int(pubkey[1])
print("Uncompressed pubkey: ", uncompressed.hex())

# prefix of 0x02 or 0x03 depending on the evenness of y coordinate, followed by x coordinate
x_coord, y_coord = pubkey[0], pubkey[1]
prefix = b"\x02" if y_coord % 2 == 0 else b"\x03"
compressed = prefix + bytes_from_int(x_coord)
print("Compressed pubkey: ", compressed.hex())

Uncompressed pubkey:  0479be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8
Compressed pubkey:  0279be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798


#### Taproot x-only public keys
Since taproot, a further step has been made to reduce the on-chain size of public keys by removing the `0x02`/`0x03` prefix from the public key and instead assuming that the y-coordinate has an even value. This is the equivalent to assuming all public keys would have had the `0x02` prefix, which means we do not need to include it and therefore just use the 32-byte x coordinate. For a detailed explanation see [Jonas Nick's blog post](https://medium.com/blockstream/reducing-bitcoin-transaction-sizes-with-x-only-pubkeys-f86476af05d7) explaining it.

#### Negating secret keys
Having the requirement that the y-coordinate must have even value leads to the question of what we do with secret keys that generate a pubkey with an odd y-coordinate. Do we just not use those secret keys? Fortunately, we can take advantage of the properties of Secp256k1 to give us a new secret key that has a corresponding public key with the same x-coordinate but an even y-coordinate. We do this by negating the secret key:

`(SECP256K1 - privkey_int) % SECP256K1` 

Here's an example of negating a private key so that we end up with a public key with the same x-coordinate but an even y-coordinate.

In [5]:
# This private key has been chosen as it'll produce a public key with an odd y-coordinate
original_privkey = bytes.fromhex("0000000000000000000000000000000000000000000000000000000000000006")
original_privkey_int = int_from_bytes(original_privkey)

original_pubkey = point_mul(G, original_privkey_int)
print("original_pubkey: ", original_pubkey)
odd_or_even = "even" if original_pubkey[1] % 2 == 0 else "odd"
print(f"y coordinate is {odd_or_even}\n")

print("Negating the private key..")
negated_privkey_int = (SECP256K1_ORDER - original_privkey_int) % SECP256K1_ORDER
new_pubkey = point_mul(G, negated_privkey_int)
print("new_pubkey: ", new_pubkey)
odd_or_even = "even" if new_pubkey[1] % 2 == 0 else "odd"
print(f"y coordinate is {odd_or_even}")

original_pubkey:  (115780575977492633039504758427830329241728645270042306223540962614150928364886, 78735063515800386211891312544505775871260717697865196436804966483607426560663)
y coordinate is odd

Negating the private key..
new_pubkey:  (115780575977492633039504758427830329241728645270042306223540962614150928364886, 37057025721515809211679672464182131982009266967775367602652617524301408111000)
y coordinate is even


## Linear property of elliptic curves: 

Understanding this simple concept will make a lot of the taproot features much easier to understand.

> The sum of private keys generates the same public key that is the sum of public keys.

For example, given two private keys, d<sub>A</sub> and d<sub>B</sub>, and their corresponding public keys, P<sub>A</sub> and P<sub>B</sub>:<br>
G\*d<sub>A</sub> = P<sub>A</sub><br>
G\*d<sub>B</sub> = P<sub>B</sub>

Summing the private keys and producing a public key produces the same result as summing their public keys:

(d<sub>A</sub>+d<sub>B</sub>)\*G = P<sub>A+B</sub>

This property has used in various places in bitcoin including:
- Deriving public keys with an extended pubkey.
- Securely using a pool to generate a custom vanity addresses ([video explanation](https://www.youtube.com/watch?v=6u1XEucEJ_A&t=322s) by Andreas Antonopoulos).
- TapTweaks (Chapter XX)
- Musig

In [6]:
# Define private keys and convert to int
privkey_a = bytes.fromhex("0000000000000000000000000000000000000000000000000000000000000001")
privkey_b = bytes.fromhex("0000000000000000000000000000000000000000000000000000000000000002")
privkey_a_int = int_from_bytes(privkey_a)
privkey_b_int = int_from_bytes(privkey_b)

privkey_ab_int = (privkey_a_int + privkey_b_int) % SECP256K1_ORDER
pubkey_ab = point_mul(G, privkey_ab_int)
print("Pubkey generated from summing private keys: ", pubkey_ab)

pubkey_a = point_mul(G, privkey_a_int)
pubkey_b = point_mul(G, privkey_b_int)

pubkey2 = point_add(pubkey_a, pubkey_b)
print("Pubkey generated from summing public keys: ", pubkey2)

assert(pubkey_ab == pubkey2)
print("Success!")

Pubkey generated from summing private keys:  (112711660439710606056748659173929673102114977341539408544630613555209775888121, 25583027980570883691656905877401976406448868254816295069919888960541586679410)
Pubkey generated from summing public keys:  (112711660439710606056748659173929673102114977341539408544630613555209775888121, 25583027980570883691656905877401976406448868254816295069919888960541586679410)
Success!
